In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from sklearn import metrics
import numpy as npw
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.preprocessing import MinMaxScaler





# Read the CSV file
data = pd.read_csv("../data/data/aapl_raw_data.csv")
data.shape
data.tail(1)

data = data.iloc[:10747]

data.tail(1)


data.isnull().sum()
data=data.fillna(0)  # Filling null values with zero
data.isnull().sum()



#data["date"] = data["date"].astype(float)
data["open"] = data["open"].astype(float)
data["high"] = data["high"].astype(float)
data["low"] = data["low"].astype(float)
data["volume"] = data["volume"].astype(float)
data["adjusted_close"] = data["adjusted_close"].astype(float)
data["change_percent"] = data["change_percent"].astype(float)
data["avg_vol_20d"] = data["avg_vol_20d"].astype(float)

data["close"] = data["close"].astype(float)

#print(data.tail(1))

In [2]:
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Convert data to numpy arrays
x_data = data[['open', 'high', 'low', 'volume', 'adjusted_close', 'change_percent', 'avg_vol_20d']].values
y_data = data["close"].values.reshape(-1, 1)  # Reshape y_data if it's 1D

# Initialize the scalers
scaler_x = StandardScaler()
scaler_y = StandardScaler()

# Fit the scaler to the training data and transform both training and test data
x_data_scaled = scaler_x.fit_transform(x_data)
y_data_scaled = scaler_y.fit_transform(y_data)

# Convert scaled data to Tensors
x_feature_tensors = torch.tensor(x_data_scaled, dtype=torch.float32)
y_feature_tensors = torch.tensor(y_data_scaled, dtype=torch.float32)

# Split the training data into training and temporary sets
x_train, x_temp, y_train, y_temp = train_test_split(x_feature_tensors, y_feature_tensors, test_size=0.2, random_state=42)

# Split the temporary set into validation and test sets
x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=0.5, random_state=42)

# Convert scaled labels back to numpy arrays
y_train = y_train.numpy()
y_val = y_val.numpy()
y_test = y_test.numpy()



# Now you can use x_train_scaled, x_val, x_test, y_train_scaled, y_val_scaled, y_test_scaled for your model training


In [3]:

learning_rate = 0.001
window_size = 50
test_window_size = 50
hidden_dim = 64
n_layers = 4
batch_evaluation_frequency = 10
epochs = len(x_train) - window_size
batch_size = 1
input_size = x_train.shape[1]  # Input size based on your dataset
output_size = 1  # Output size (for regression task)

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_dim, n_layers, output_size):
        super(LSTMModel, self).__init__()

        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        self.lstm = nn.LSTM(input_size, hidden_dim, n_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_size)

    def forward(self, x, hidden):
        batch_size = x.size(0)
        out, hidden = self.lstm(x, hidden)
        out = self.fc(out[:, -1, :])
        return out, hidden

    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        return hidden


In [4]:
# Initialize the LSTM model
model = LSTMModel(input_size=input_size, hidden_dim=hidden_dim, n_layers=n_layers, output_size=output_size)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

window_size = 10
epochs = 1
# Training loop
for epoch in range(epochs):
    # Walk-Forward Validation
    for i in range(len(x_train) - window_size):
        # Initialize hidden state for each iteration
        hidden = model.init_hidden(batch_size)

        # Prepare the input and target sequences
        x_sequence = x_train[i:i + window_size]
        y_true = y_train[i + window_size:i + window_size + 1]

        # Convert sequences to PyTorch Tensors
        x_sequence = torch.unsqueeze(x_sequence, 0)
        y_true = torch.tensor(y_true, dtype=torch.float32)

        # Forward pass
        y_pred, hidden = model(x_sequence, hidden)
        loss = criterion(y_pred, y_true)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print training information
        if (i + 1) % batch_evaluation_frequency == 0:
            print(f'Epoch [{epoch + 1}/{epochs}], Step [{i + 1}/{len(x_train) - window_size}], Loss: {loss.item():.4f}')


# Validation
model.eval()
val_losses = []

with torch.no_grad():
    for i in range(len(x_val) - window_size):
        x_val_sequence = x_val[i:i + window_size]
        y_val_true = y_val[i + window_size:i + window_size + 1]

        x_val_sequence = torch.unsqueeze(x_val_sequence, 0)
        y_val_true = torch.tensor(y_val_true, dtype=torch.float32)

        y_val_pred, _ = model(x_val_sequence, hidden)
        val_loss = criterion(y_val_pred, y_val_true)
        val_losses.append(val_loss.item())

# Calculate and print the mean validation loss
mean_val_loss = np.mean(val_losses)
print(f'Mean Validation Loss: {mean_val_loss:.4f}')


Epoch [1/1], Step [10/8587], Loss: 0.0000
Epoch [1/1], Step [20/8587], Loss: 0.0000
Epoch [1/1], Step [30/8587], Loss: 0.0023
Epoch [1/1], Step [40/8587], Loss: 0.3514
Epoch [1/1], Step [50/8587], Loss: 0.3130
Epoch [1/1], Step [60/8587], Loss: 0.3411
Epoch [1/1], Step [70/8587], Loss: 0.4858
Epoch [1/1], Step [80/8587], Loss: 0.5881
Epoch [1/1], Step [90/8587], Loss: 0.0832
Epoch [1/1], Step [100/8587], Loss: 0.0226
Epoch [1/1], Step [110/8587], Loss: 0.5159
Epoch [1/1], Step [120/8587], Loss: 0.1984
Epoch [1/1], Step [130/8587], Loss: 0.5195
Epoch [1/1], Step [140/8587], Loss: 0.5664
Epoch [1/1], Step [150/8587], Loss: 0.0289
Epoch [1/1], Step [160/8587], Loss: 0.2892
Epoch [1/1], Step [170/8587], Loss: 4.7869
Epoch [1/1], Step [180/8587], Loss: 0.0366
Epoch [1/1], Step [190/8587], Loss: 0.3257
Epoch [1/1], Step [200/8587], Loss: 0.1999
Epoch [1/1], Step [210/8587], Loss: 0.0235
Epoch [1/1], Step [220/8587], Loss: 0.1390
Epoch [1/1], Step [230/8587], Loss: 0.4613
Epoch [1/1], Step [2

KeyboardInterrupt: 

In [ ]:
# Assume new_data is your input data for the next day
# Make sure to preprocess and scale new_data appropriately

# Convert new_data to a PyTorch tensor
new_data_scaled = scaler_x.transform(new_data)
new_data_tensor = torch.tensor(new_data_scaled, dtype=torch.float32)

# Set the model to evaluation mode
model.eval()

# Initialize hidden state
hidden = model.init_hidden(1)  # Assuming batch size is 1

# Forward pass
with torch.no_grad():
    y_pred, _ = model(new_data_tensor.unsqueeze(0), hidden)

# Inverse transform the predicted close value
predicted_close_scaled = y_pred.item()
predicted_close = scaler_y.inverse_transform(np.array([[predicted_close_scaled]]))

print(f'Predicted Close Value for the Next Day: {predicted_close[0, 0]:.4f}')


NameError: name 'new_data' is not defined


can you predict the "close value" for the next day after the existing data using the trained method? I dont have input for the future. I have to create it. 

In [ ]:
# Print the last 50 values of x_train
print("Last 50 values of x_train:")
print(x_train[-50:].numpy())

# Print the last window
print("\nLast window (x_sequence):")
print(x_sequence[0].numpy())


Last 50 values of x_train:
[[-3.8239661e-01 -3.6861533e-01 -3.8869005e-01 -1.2194055e-01
  -4.4869837e-01 -4.7490220e+00 -5.5267133e-02]
 [ 1.3212335e+00  1.3480754e+00  1.3426359e+00  3.5530314e-01
  -2.3301007e-01  1.0631542e-01  7.1244866e-01]
 [ 2.8130737e-01  2.7440774e-01  2.7848962e-01  3.2699594e+00
   3.3431654e+00 -1.9936468e-01 -8.7569481e-01]
 [ 3.7296879e+00  3.7091699e+00  3.7638538e+00 -4.2365277e-01
   3.8722478e-02  5.2999120e-02 -3.0757835e-02]
 [ 2.5817108e-01  2.6726300e-01  2.6891381e-01  1.9915811e+00
   3.3593824e+00  6.9990349e-01 -9.1779560e-01]
 [ 2.6575676e-01  2.7613753e-01  2.7013949e-01  2.1285069e+00
   3.3952315e+00  4.8663834e-01 -8.5606867e-01]
 [ 1.6491501e-02  1.4939867e-02  2.5764246e-02  8.5789776e-01
   2.1987750e-01  5.0441045e-01 -4.7077781e-01]
 [ 2.9192731e-01  3.1374159e-01  3.0568498e-01  2.4002156e+00
   3.5360012e+00  6.4303279e-01 -9.1319573e-01]
 [-6.6645885e-01 -6.6945368e-01 -6.6689479e-01 -5.9105599e-01
  -4.5095354e-01 -7.3608208e-01